# Ce notebook rassemble les méthodes statistiques employées pour la sélection de variables pour le modèle de régression logistique ordinale

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from mord import LogisticAT
from typing import List
import numpy as np

from sklearn.metrics import accuracy_score
from mord import LogisticAT

import pickle

In [2]:
#df_train = pd.read_csv("../../data/train.csv")
#df_test = pd.read_csv("../../data/test.csv")
df_train = pd.read_csv("C://Cours M2 S1//Conférences, SAS, VBA, Gestion de projet//Gestion de projet digital//train.csv")
df_test = pd.read_csv("C://Cours M2 S1//Conférences, SAS, VBA, Gestion de projet//Gestion de projet digital//test.csv")

In [3]:
def get_train_test_sets(train_data, test_data):
    """Fonction qui récupère les dataframes train et test et retourne les vecteurs utiles pour la phase
    d'entrainement et test.

    Args:
        train_data (DataFrame): données d'entrainement pré-traitées
        test_data (DataFrame): données de test pré-traitées

    Returns:
        tuple: X_train, X_test, y_train, y_test
    """
    
    X_train = train_data.drop(labels=["score"], axis=1).values
    y_train = train_data["score"].values

    X_test = test_data.drop(labels=["score"], axis=1).values
    y_test = test_data["score"].values
    
    ## On normalise les données
    from sklearn.preprocessing import StandardScaler
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [4]:
X_train, X_test, y_train, y_test = get_train_test_sets(train_data=df_train, test_data=df_test)

# Le modèle de régression logistique ordinale initial

### On regarde les coefficients et l'erreur de classification du modèle global initial

In [5]:
#On entraîne le modèle initial

def model_training(X_train, X_test, y_train, y_test):
    """Fonction pour entrainer le modèle de régression logistique ordinale

    Args:
        X_train (Array): 
        X_test (Array): 
        y_train (Array): 
        y_test (Array): 
    """

    # Maintenant, on utilise les variables sélectionnées pour la régression logistique ordinale avec mord
    model_ordinal = LogisticAT(alpha=0)  # On utilise alpha=0 pour ne pas faire de régularisation

    # On entraîne le modèle sur les données d'entraînement avec les variables sélectionnées
    model_ordinal.fit(X_train, y_train)

    # prédictions sur les données de test avec les variables sélectionnées
    predictions_ordinal = model_ordinal.predict(X_test)

    # Calcul de l'exactitude du modèle de régression logistique ordinale
    accuracy_ordinal = accuracy_score(y_test, predictions_ordinal)
    
    print("Coefficients du modèle de régression logistique :")
    print(model_ordinal.coef_)
   
    print("Précision du modèle sur les données de test : {:.2f}%".format(accuracy_ordinal * 100))

    return

In [6]:
#On l'applique et on montre ses coefficients et son erreur de classification
model_training(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

Coefficients du modèle de régression logistique :
[-8.69431442e-01 -8.68057631e-01 -2.52819023e+00 -6.25124049e-01
 -1.17822450e+00 -2.90415631e-02 -2.14414883e+00  1.16671329e+00
  3.17895520e-01 -1.00617077e+02  9.87501392e+01 -3.81371512e-02
 -7.46649800e-02 -3.59015466e-01  1.16666886e+00]
Précision du modèle sur les données de test : 73.92%


## Méthodes de sélection de variables

## *1-* Sélection par régression Lasso

LogisticRegressionCV est utilisé pour effectuer une régression logistique avec une validation croisée pour sélectionner le meilleur paramètre de régularisation (C) parmi une liste de valeurs candidates ()

In [7]:
def selection_lasso(train_data, test_data, max_iter = 1000, seuil=0.2):
    """Fonction pour faire la sélection de variables en se basant sur un modèle Lasso.

    Args:
        train_data (Dataframe): données préparées pour le train
        test_data (DataFrame): données préparées pour le test
        max_iter (int, optional): Nombre d'itérations pour obtenir la convergence, par défaut 1000
        seuil (float, optional): Valeur seuil en valeur absolur au dessus de laquelle on sélectionn les variables. Par défaut 0.2.

    Returns:
        tuple: variables_lasso (Liste) noms des variables choisies, X_train_final (Array), X_test_final (Array), y_train(Array), y_test (Array)
        vecteurs des variables choisies
    """
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.feature_selection import SelectFromModel
    
    X_train, X_test, y_train, y_test = get_train_test_sets(train_data=train_data, test_data=test_data)
    


    # modèle de régression logistique avec pénalité L1 (LASSO)
    model_lasso = LogisticRegression(penalty='l1', solver='saga', max_iter=max_iter, random_state=None)

    # SelectFromModel pour récupérer les paramètres du modèle
    sfm = SelectFromModel(estimator=model_lasso)
    sfm.fit(X_train, y_train)

    # indices des variables sélectionnées
    indices_lasso = sfm.get_support(indices=True)
    
    # noms des variables correspondantes dans les données
    variables = [train_data.columns[i] for i in indices_lasso]
    
    # coefficients des variables sélectionnées
    coefficients_lasso = sfm.estimator_.coef_.flatten()

    # noms et coefficients des variables
    print("#########################################################")
    print()
    print("Coefficients obtenus après Lasso:")
    for variable, coefficient in zip(variables, coefficients_lasso):
        print(f"{variable}: {coefficient}")

    # On pénalise en ne prenant que les coeffs différents au dessus du seuil passé en argument
    indices_choisis = [i for i, coef in zip(indices_lasso, coefficients_lasso) if abs(coef) > seuil]

    # variables correspondantes aux indices choisies
    variables_lasso = [train_data.columns[i] for i in indices_choisis]

    print("##########################################################")
    print()
    print("Variables sélectionnées après Lasso")
    print()
    for variable in variables_lasso:
        print(f"{variable}")


    # on utilise les indices filtrés pour obtenir les données des variables optimales sélectionnées
    X_train_final = X_train[:, indices_choisis]
    X_test_final = X_test[:, indices_choisis]
    
    return variables_lasso, X_train_final, X_test_final, y_train, y_test 


On effectue la sélection de variables

In [8]:
_, X_train_lasso, X_test_lasso, y_train, y_test = selection_lasso(train_data=df_train, test_data=df_test)

#########################################################

Coefficients obtenus après Lasso:
energy_100g: 1.480942639366321
fat_100g: 1.1784743246135954
saturated-fat_100g: 5.0696470696560825
trans-fat_100g: 1.5001631855858724
cholesterol_100g: 1.737175704050646
carbohydrates_100g: 0.6348745186366846
sugars_100g: 2.9465936128379755
fiber_100g: -1.958085840128126
proteins_100g: 0.0
salt_100g: 1.2795379031110787
sodium_100g: 1.2801899356246484
vitamin-a_100g: 0.8938106508001012
vitamin-c_100g: 0.5018054579698419
calcium_100g: 0.0
iron_100g: -7.487952521369647
##########################################################

Variables sélectionnées après Lasso

energy_100g
fat_100g
saturated-fat_100g
trans-fat_100g
cholesterol_100g
carbohydrates_100g
sugars_100g
fiber_100g
salt_100g
sodium_100g
vitamin-a_100g
vitamin-c_100g
iron_100g


C:\Users\alexa\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [15]:
def model_training(X_train, X_test, y_train, y_test, selection_method = 'lasso'):
    """Fonction pour entrainer le modèle de régression logistique ordinale

    Args:
        X_train (Array): 
        X_test (Array): 
        y_train (Array): 
        y_test (Array): 
    """

    # Maintenant, on utilise les variables sélectionnées pour la régression logistique ordinale avec mord
    model_ordinal = LogisticAT(alpha=0)  # On utilise alpha=0 pour ne pas faire de régularisation

    # On entraîne le modèle sur les données d'entraînement avec les variables sélectionnées
    model_ordinal.fit(X_train, y_train)

    # prédictions sur les données de test avec les variables sélectionnées
    predictions_ordinal = model_ordinal.predict(X_test)

    # Calcul de l'exactitude du modèle de régression logistique ordinale
    accuracy_ordinal = accuracy_score(y_test, predictions_ordinal)

    print()
    print("Précision du modèle : {:.2f}%".format(accuracy_ordinal * 100))
    return 


On entraine le modèle sur les variables sélectionnées

In [16]:
model_training(X_train=X_train_lasso, X_test=X_test_lasso, y_train=y_train, y_test=y_test)


Précision du modèle : 72.58%


## *2-* Sélection par Random Forest

In [12]:
def random_forest_selection(train_data, test_data, expected_nb_var, nb_estimator =100):
    """Fonction pour faire de la sélection de variables en fonction de l'importance des variables obtenues en
    utilisant une modèle de RandomForestClassifier

    Args:
        train_data (DataFrame): dataframe d'entrainement
        test_data (DataFrame): dataframe de test
        expected_nb_var (int): nombre de variables qu'on souhaite obtenir
        nb_estimator (int) : nombres d'arbres dans la forêt
    """

    # on obtient les matrices (train & test) à passer dans le modèle
    X_train, X_test, y_train, y_test = get_train_test_sets(train_data=train_data, test_data=test_data)
    
    from sklearn.ensemble import RandomForestClassifier
    

    # modèle Random Forest avec bootstrap

    rf_model = RandomForestClassifier(n_estimators=nb_estimator, random_state=None, bootstrap=True, criterion='gini')
    """ n-estimators : On aura donc 100 arbres qui seront tous légèrement différents
    
        bootstrap : construire de nouveaux échantillons par tirage aléatoire avec remise
        
        criterion = 'gini' : L'indice de Gini mesure l'impureté d'un nœud (une division) 
        dans un arbre de décision. Plus précisément, il évalue à quel point un nœud est mélangé en termes de classes de la cible
        
    
    """

    # Entraînement du modèle sur les données
    rf_model.fit(X_train, y_train)

    # Importance des variables à partir du modèle
    feature_importances = rf_model.feature_importances_

    # Sélection des indices des variables importantes (les nvar premières)
    important_indices = feature_importances.argsort()[-expected_nb_var:][::-1]
    
    variables_rf = [train_data.columns[i] for i in important_indices]
    
    print("##########################################################")
    print()
    print("Variables sélectionnées après Random Forest")
    print()
    for variable in variables_rf:
        print(f"{variable}")

    # Matrices d'entrainement et de test pour entrainer le modèle logistique
    X_train_rf = X_train[:, important_indices]
    X_test_rf = X_test[:, important_indices]
    
    return variables_rf, X_train_rf, X_test_rf, y_train, y_test



In [13]:
_, X_train_rf, X_test_rf, y_train, y_test = random_forest_selection(train_data=df_train, test_data=df_test, nvar=10)

##########################################################

Variables sélectionnées après Random Forest

nutrition-score-fr_100g
trans-fat_100g
saturated-fat_100g
calcium_100g
fat_100g
cholesterol_100g
energy_100g
code
sugars_100g
vitamin-c_100g
